### Sumário

1.   **Facebook**

        1.1 Interaction in groups

        1.2 Pokes

        1.3 Pages follow 

        1.4 Groups + pages

        1.5 Posts
        
        1.6 Comments





### Imports and instalations

In [1]:
!pip install wordcloud -q 

In [2]:
import pandas as pd
import json
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator 
from PIL import Image
import matplotlib.pyplot as plt
from unicodedata import normalize
import numpy as np

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. Facebook

Período analisado: 1 de Jan de 2013 - 1 de Jan de 2018.


## 1.1 Interaction in groups 

In [9]:
path_group_interacion = '/content/drive/MyDrive/Case Ifood/FACEBOOK - 1/activity_messages/group_interactions.json'

In [10]:
group_interaction = pd.read_json(path_group_interacion)

In [11]:
group_interaction

,group_interactions_v2
0,"{'name': '', 'description': '', 'entries': [{'..."


In [ ]:
# nivelando
entries = pd.json_normalize(group_interaction['group_interactions_v2'])['entries']

In [ ]:

entries.keys()

In [ ]:
# arquivo nivelado depois do tratamento realiza acima
df_group_interacion = pd.json_normalize(entries[0])
df_group_interacion

In [ ]:
# tornando os nomes das legendas mais intuitivos em relação ao que representam:
df_group_interacion.rename(columns={'data.name': 'name_group',
                                   'data.value': 'value_interaction',
                                   'data.uri': 'uri_group'}, inplace = True)


In [ ]:
df_group_interacion

In [ ]:
# analisando cada campo: necessário conformar 'value_interaction'
df_group_interacion.info()

In [ ]:
# para conformar a coluna 'value_interaction', é necessário separar a quantidade de vezes da palavra vezes/vez.
# selecionando a coluna
value_interactions = df_group_interacion['value_interaction']

In [ ]:
# isolando somente a quantidade de vezes e adicionando em
# uma lista para posteriormente adicionar como nova serie ao DataFrame

value_split = value_interactions.str.split(' ')
interactions = []
for v in value_split:
  v = v[0]
  interactions.append(v)
  
print(interactions)

In [ ]:
# criando nova coluna somente com os valores filtrados da coluna 'value_interaction'
df_group_interacion['value_interactions'] = pd.DataFrame(interactions)


In [ ]:
df_group_interacion['value_interactions'] = pd.to_numeric(df_group_interacion['value_interactions'])

In [ ]:
# eliminando antiga coluna em que a quantidade de vezes estava como object e como a palavra
df_group_interacion.drop(columns=['value_interaction'], inplace=True)


In [ ]:
df_group_interacion

In [ ]:
df_group_interacion.info()

## 1.2 Pokes

ações de cutucadas. 

In [ ]:
path_pokes = '/content/drive/MyDrive/Case Ifood/FACEBOOK - 1/other_activity/pokes.json'

In [ ]:
# Leitura do json com dados:
with open(path_pokes) as access_json:
  read_pokes = json.load(access_json, encoding='latin-1')

In [ ]:
# acessando(entrando) no dicinário:
access_pokes = read_pokes['pokes_v2']


In [ ]:
# acessando a lista de dicinários contida dentro do 'pokes_v2':
access_activity = access_pokes['activity_log_data']

In [ ]:
# função para acessar a lista de dicionários de forma otimizada:
def get_pokes(): 

  access_pokes = read_pokes['pokes_v2']

  for pokes_data in access_pokes:
    access_activity = pokes_data['activity_log_data']
 
      

In [ ]:
# transformando o json em um dataframe
df_pokes = pd.json_normalize(access_activity, record_path= ['data'], meta = ['timestamp', 'title'])
df_pokes['name'].value_counts()

In [ ]:
# Alterando os campos da serie 'title' para 'Cutucou'
df_pokes.loc[:, 'title'] = 'Cutuquei'

In [ ]:
# renomeando colunas
df_pokes.rename(columns={'name': 'name_user',
                                   'title': 'action',}, inplace = True)

In [ ]:
df_pokes.info()

## 1.3 Pages follow
Páginas que sigo na rede social.

In [ ]:
path_pages_follow = '/content/drive/MyDrive/Case Ifood/FACEBOOK - 1/pages/pages_you_follow.json'

In [ ]:
with open(path_pages_follow) as access_json:
  read_pages_follow = json.load(access_json, encoding ='latin-1')

In [ ]:
read_pages = read_pages_follow['pages_followed_v2']

In [ ]:
df = pd.json_normalize(read_pages)
df.explode('data')

In [ ]:
df_pages_follow = pd.json_normalize(read_pages, record_path= ['data'], meta = ['timestamp', 'title'])
df_pages_follow

In [ ]:
df_pages_follow.info()

## 1.4 Groups + pages
Nuvem de palavras através da união de grupos e páginas que participo/segui para demonstrar os interesses que mais se destacam. 

In [ ]:
# recuperando dataframe sobre grupos e retirando colunas que não interessam e renomeando a que restou como 'interests'
groups = df_group_interacion.drop(columns=['uri_group', 'value_interactions'])
groups = groups.rename(columns={'name_group': 'interests'})

In [ ]:
# realizando a mesma rotina anterior: é necessário deixar as colunas com o mesmo nome para ser viável concater
pages = df_pages_follow.drop(columns=['timestamp', 'title'])
pages = pages.rename(columns={'name': 'interests'})
pages

In [ ]:
# unindo ambas as series 
groups_and_pages = pd.concat([groups, pages])

In [ ]:
groups_and_pages

In [ ]:
groups_and_pages.to_csv('group_and_pages.csv', encoding='latin-1', index= False, header = True)


In [ ]:
df_group_and_pages = pd.read_csv('/content/df_group_and_pages.csv')

In [ ]:
# armazenando todas as palavras da serie 'interests' em minusculo
palavras = ' '.join(df_group_and_pages['interests'].str.lower())
# conferindo a quantidade de palavras
len(palavras)

In [ ]:
# definindo palavras que devem ser ignoradas
stopwords = set(STOPWORDS)
stopwords.update(['e', 'de', 'da', 'dos','o', 'a' 'os', 'você', 'são', 'sem', 'ao', 'é', 'salgueiro', 'brasil', 'ana', 'oliveira', 'que', 'oficial', 'depressão', 'pe', 'mais'])

In [ ]:
# criando a nuvem de palavras
wordcloud = WordCloud(stopwords=stopwords,
                      background_color = 'black',
                      width=1600, height= 800).generate(palavras)

In [ ]:
# plotando nuvem de palavras
fix, ax = plt.subplots()
ax.imshow(wordcloud, interpolation = 'bilinear')
ax.set_axis_off()

plt.imshow(wordcloud);
wordcloud.to_file('interests_facebook.png')

In [ ]:
mask_facebook = np.array(Image.open('/content/drive/MyDrive/Case Ifood/FACEBOOK - 1/Facebook-Logo-1.jpg'))

# criando a nuvem de palavras
wordcloud = WordCloud(stopwords=stopwords,
                      background_color = 'White', colormap = 'Blues',
                      width= 3000, height= 2500, max_words= 2000,
                      max_font_size = 500, min_font_size = 30, 
                      mask = mask_facebook).generate(palavras)

In [ ]:
# Plotando nuvem de palavras novamente, com a mascara
fix, ax = plt.subplots()
ax.imshow(wordcloud, interpolation = 'bilinear')
ax.set_axis_off()

plt.imshow(wordcloud);
wordcloud.to_file('nuvem_facebook.png')

## 1.5 Posts

Publicações na rede social.

In [ ]:
path_posts = '/content/drive/MyDrive/Case Ifood/FACEBOOK - 1/posts/your_posts_1.json'

In [ ]:
# Leitura do json
with open(path_posts) as access_json:
  read_posts = json.load(access_json)

In [ ]:

df_posts = pd.json_normalize(read_posts)
df_posts = df_posts.explode('data')
df_posts = df_posts.explode('tags')

In [ ]:
df_posts

In [ ]:
# Todas as linhas apresentam timestamp, devido isso considero que os 'NaN' que aparecem nas colunas 'title' e 'post'
# pode indicar que a publicação foi excluída. Nesse cenário, considerarei todas como registro de posts. 

df_posts.info()

In [ ]:
df_posts = pd.json_normalize(read_posts, record_path= ['data'], meta = ['timestamp', 'tags', 'title', 'attachments'], errors='ignore')
df_posts

In [ ]:
# Eliminando colunas que não fazem sentido para o objetivo
df_posts.drop(columns={'update_timestamp', 'tags', 'attachments'}, inplace=True)

In [ ]:
df_posts.info()

In [ ]:
# conformando campo
df_posts['timestamp'] = pd.to_datetime(df_posts['timestamp'], unit='s')


In [ ]:
# extraindo ano do campo timestamp
df_posts['year'] = pd.DatetimeIndex(df_posts['timestamp']).year


In [ ]:
df_posts

In [ ]:
# depois de extrair o que era necessário, eliminando coluna
df_posts.drop(columns={'timestamp'}, inplace=True)

In [ ]:
df_posts

In [ ]:
# preenchendo campos nulos(compreende-se que houve a postagem, mas, por exemplo, o post pode ter sido excluido pela fonte.)
df_posts = df_posts.fillna('Postagem indisponível')


In [ ]:
df_posts

In [ ]:
# exportando DataFrame
df_posts.to_csv( 'posts_facebook_nat_ok.csv' , index = False , header = True, encoding='latin-1')


## 1.6 Comments

In [ ]:
path_comments = '/content/drive/MyDrive/Case Ifood/FACEBOOK - 1/comments.json'

In [ ]:
# leitura do json
with open(path_comments) as access_json:
  read_comments = json.load(access_json, encoding ='UFT-8')

In [ ]:
read_comments = read_comments['comments_v2']


In [ ]:
df_comments = pd.json_normalize(read_comments)

In [ ]:
df_comments = pd.json_normalize(read_comments, meta_prefix=['data'])


In [ ]:
df_comments.drop(columns={'data', 'attachments'}, inplace=True)


In [ ]:
df_comments

In [ ]:
df_comments.info()

In [ ]:
# conformando campo
df_comments['timestamp'] = pd.to_datetime(df_comments['timestamp'], unit='s')


In [ ]:
# extraindo ano 
df_comments['year'] = pd.DatetimeIndex(df_comments['timestamp']).year
df_comments

In [ ]:
# eliminando coluna que agora já é dispensavel
df_comments.drop('timestamp', axis=1, inplace=True)


In [ ]:
df_comments.rename(columns={'title': 'comment_post'})

In [ ]:
# exportando
df_comments.to_csv('comments_facebook.csv', encoding='utf-8', index = False, header = True)
